In [1]:
import os 
import torch
import joblib
import scanpy as sc 
import numpy as np
import pandas as pd 
from sklearn.neighbors import kneighbors_graph
import scipy.sparse as sp
from torch_geometric.data import Data

data_root = '../data/melanoma/'
res_path = '../results/sagpool/Tuning_hd_256_convtype_GCN_sag_r_0.015625_lsim_0.5_ldiff_0.01_act_op_relu_K_2_bl_1_al_1'
UTAG_path = os.path.join(data_root,'adata','utag_results_dist10_leiden03.h5ad')
adata = sc.read(UTAG_path)
domain_column = 'UTAG Label_leiden_0.3'
visualize_path = os.path.join(data_root,'vis_cell_type')
gnn_data_path = os.path.join(data_root,'gnn_data')
node_mask_path = os.path.join(res_path,'node_mask_with_channel_attention')
label_path = os.path.join(data_root,'fold_train_val','OS_label_dict.pkl')
EXP_channel_number = 35

In [3]:
np.unique(adata.obs[domain_column].to_numpy(),return_counts=True)

(array(['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '2', '3',
        '4', '5', '6', '7', '8', '9'], dtype=object),
 array([184209, 127979,   4539,   4106,   2166,   1539,    796,    611,
           510,    112, 105380,  37701,  33157,  29188,  21534,  11414,
          8932,   5951]))

In [ ]:
domain_id,domain_cell_num = np.unique(adata.obs[domain_column].to_numpy(),return_counts=True)

In [4]:
adata.obs

,roi,cell_type,cell_type_final,UTAG Label_leiden_0.3
0,66_ROI9,Lymphocyte,Double-positive T cells,1
1,66_ROI9,Lymphocyte,B cells,1
2,66_ROI9,Stroma,Stroma (Collagen+),1
3,66_ROI9,Lymphocyte,B cells,7
4,66_ROI9,Lymphocyte,Regulatory T cells,1
...,...,...,...,...
579819,68_ROI13,Stroma,Stroma (Collagen+),6
579820,68_ROI13,Tumor,Tumor (CA9+),0
579821,68_ROI13,Stroma,Stroma (Vimentin+),0
579822,68_ROI13,Stroma,Stroma (Collagen+),0


In [5]:
PD_ROI_cut = [0]
for i in range(1,len(adata.obs)):
    if adata.obs.roi[i]!=adata.obs.roi[i-1]:
        PD_ROI_cut.append(i)
PD_ROI_cut.append(len(adata.obs))
print(len(PD_ROI_cut))

147


In [6]:

g_cell_list = []
for i in range(len(PD_ROI_cut)-1):
    ROI_name = adata.obs['roi'][PD_ROI_cut[i]] 
    g = torch.load(os.path.join(gnn_data_path,ROI_name+'.pkl'))
    g_cell_list = g_cell_list + g.cell_type_final

In [7]:
adata.obs['cell_type_final']=g_cell_list

In [8]:
adata.obs

,roi,cell_type,cell_type_final,UTAG Label_leiden_0.3
0,66_ROI9,Lymphocyte,Double-positive T cells,1
1,66_ROI9,Lymphocyte,B cells,1
2,66_ROI9,Stroma,Stroma (Collagen+),1
3,66_ROI9,Lymphocyte,B cells,7
4,66_ROI9,Lymphocyte,Regulatory T cells,1
...,...,...,...,...
579819,68_ROI13,Stroma,Stroma (Collagen+),6
579820,68_ROI13,Tumor,Tumor (CA9+),0
579821,68_ROI13,Stroma,Stroma (Vimentin+),0
579822,68_ROI13,Stroma,Stroma (Collagen+),0


In [ ]:
adata.obs.groupby([domain_column,"cell_type_final"]).describe().unstack().to_csv(os.path.join(res_path,'domain_cell_type_melanoma.csv'))


计算每个domain的HighAttention部分的细胞个数

In [8]:
def Jet(score_color):
    if score_color<=31:
        return (int(128+4*score_color),0,0)
    elif score_color==32:
        return (255,0,0)
    elif score_color>=33 and score_color<=95:
        return (255,4+4*(score_color-33),0)
    elif score_color==96:
        return (254,255,2)
    elif score_color>=97 and score_color<=158:
        return (250-4*(score_color-97),255,6+4*(score_color-97))
    elif score_color==159:
        return (1,255,254)
    elif score_color>=160 and score_color<=223:
        return (0,252-4*(score_color-160),255)
    elif score_color>=224 and score_color<=255:
        return (0,0,252-4*(score_color-224))
    print('ERROR score>255!',score_color)

In [10]:
res = {}
label_all = joblib.load(os.path.join(data_root,'label_and_fold','response_label_dict.pkl'))
os.makedirs(os.path.join(res_path,'UTAG_channel_mean_in_highlight'))
os.makedirs(os.path.join(res_path,'channel_mean_in_highlight'))
os.makedirs(os.path.join(res_path,'UTAG_channel_mean_in_whole'))

for i in range(len(PD_ROI_cut)-1):
    ROI_name = adata.obs['roi'][PD_ROI_cut[i]] 
    n_m = torch.load(os.path.join(node_mask_path,ROI_name+'.pkl')).node_mask

    # if have channel score of sharp value
    # channel_score = torch.load(os.path.join(node_mask_path,ROI_name+'.pkl')).channel_A.detach().cpu()
    # channel_score_df = pd.DataFrame(channel_score[n_m].numpy())
    # channel_score_df['utag'] = adata.obs['UTAG Label_leiden_0.3'][PD_ROI_cut[i]:PD_ROI_cut[i+1]].to_numpy()[n_m.numpy()]

    # channel_score_df.groupby('utag')[list(range(EXP_channel_number))].mean().to_csv(os.path.join(res_path,'UTAG_channel_mean_in_highlight',ROI_name+'.csv'))
    # channel_score_df[list(range(EXP_channel_number))].mean().to_csv(os.path.join(res_path,'channel_mean_in_highlight',ROI_name+'.csv'))

    # channel_score = torch.load(os.path.join(node_mask_path,ROI_name+'.pkl')).channel_A.detach().cpu()
    # channel_score_df = pd.DataFrame(channel_score.numpy())
    # channel_score_df['utag'] = adata.obs['UTAG Label_leiden_0.3'][PD_ROI_cut[i]:PD_ROI_cut[i+1]].to_numpy()
    # channel_score_df.groupby('utag')[list(range(EXP_channel_number))].mean().to_csv(os.path.join(res_path,'UTAG_channel_mean_in_whole',ROI_name+'.csv'))

    g = torch.load(os.path.join(gnn_data_path,ROI_name+'.pkl'))
    v,c = np.unique(adata.obs[domain_column][PD_ROI_cut[i]:PD_ROI_cut[i+1]].to_numpy()[n_m.numpy()],return_counts=True)
    
    res[ROI_name]={}
    for p in range(int(domain_id.max())):
        res[ROI_name][str(p)]=0
    res[ROI_name]['label']=label_all[ROI_name+'.pkl']
    for v_i in range(len(v)):
        res[ROI_name][v[v_i]] = c[v_i]

print(res)

{'66_ROI9': {'0': 6, '1': 89, '2': 141, '3': 8, '4': 0, '5': 1, '6': 0, '7': 0, '8': 0, '9': 0, '10': 0, '11': 0, '12': 0, '13': 0, '14': 3, '15': 0, '16': 0, '17': 0, 'label': 1}, '66_ROI10': {'0': 12, '1': 569, '2': 24, '3': 4, '4': 0, '5': 12, '6': 0, '7': 30, '8': 0, '9': 0, '10': 1, '11': 0, '12': 0, '13': 0, '14': 0, '15': 0, '16': 0, '17': 0, 'label': 1}, '77_ROI10': {'0': 5, '1': 106, '2': 29, '3': 23, '4': 12, '5': 0, '6': 1, '7': 31, '8': 0, '9': 0, '10': 177, '11': 0, '12': 0, '13': 0, '14': 0, '15': 0, '16': 0, '17': 0, 'label': 0}, '77_ROI11': {'0': 64, '1': 4, '2': 49, '3': 57, '4': 1, '5': 1, '6': 7, '7': 0, '8': 1, '9': 5, '10': 0, '11': 1, '12': 0, '13': 0, '14': 0, '15': 0, '16': 0, '17': 0, 'label': 0}, '12_ROI9': {'0': 94, '1': 28, '2': 82, '3': 6, '4': 135, '5': 59, '6': 0, '7': 0, '8': 0, '9': 0, '10': 0, '11': 1, '12': 0, '13': 0, '14': 0, '15': 0, '16': 1, '17': 0, 'label': 0}, '12_ROI10': {'0': 125, '1': 79, '2': 105, '3': 0, '4': 34, '5': 6, '6': 0, '7': 0, '8

In [12]:
data_res = '''{"66_ROI9": {"0": 6, "1": 89, "2": 141, "3": 8, "4": 0, "5": 1, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 3, "15": 0, "16": 0, "17": 0, "label": 1}, "66_ROI10": {"0": 12, "1": 569, "2": 24, "3": 4, "4": 0, "5": 12, "6": 0, "7": 30, "8": 0, "9": 0, "10": 1, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "77_ROI10": {"0": 5, "1": 106, "2": 29, "3": 23, "4": 12, "5": 0, "6": 1, "7": 31, "8": 0, "9": 0, "10": 177, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "77_ROI11": {"0": 64, "1": 4, "2": 49, "3": 57, "4": 1, "5": 1, "6": 7, "7": 0, "8": 1, "9": 5, "10": 0, "11": 1, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "12_ROI9": {"0": 94, "1": 28, "2": 82, "3": 6, "4": 135, "5": 59, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 1, "12": 0, "13": 0, "14": 0, "15": 0, "16": 1, "17": 0, "label": 0}, "12_ROI10": {"0": 125, "1": 79, "2": 105, "3": 0, "4": 34, "5": 6, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "12_ROI11": {"0": 58, "1": 81, "2": 107, "3": 155, "4": 117, "5": 11, "6": 2, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "12_ROI12": {"0": 179, "1": 9, "2": 22, "3": 4, "4": 33, "5": 4, "6": 2, "7": 0, "8": 0, "9": 0, "10": 0, "11": 1, "12": 0, "13": 0, "14": 0, "15": 0, "16": 2, "17": 0, "label": 0}, "59_ROI9": {"0": 3, "1": 16, "2": 1, "3": 209, "4": 2, "5": 2, "6": 0, "7": 0, "8": 1, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "59_ROI10": {"0": 26, "1": 20, "2": 9, "3": 92, "4": 0, "5": 9, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "60_ROI10": {"0": 292, "1": 7, "2": 159, "3": 15, "4": 27, "5": 33, "6": 1, "7": 0, "8": 1, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "60_ROI11": {"0": 124, "1": 23, "2": 152, "3": 107, "4": 100, "5": 0, "6": 12, "7": 0, "8": 2, "9": 1, "10": 0, "11": 13, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "60_ROI12": {"0": 19, "1": 187, "2": 239, "3": 13, "4": 38, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 8, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 1, "label": 1}, "84_ROI9": {"0": 3, "1": 121, "2": 15, "3": 0, "4": 12, "5": 0, "6": 0, "7": 697, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "84_ROI11": {"0": 0, "1": 9, "2": 0, "3": 0, "4": 0, "5": 0, "6": 0, "7": 369, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "84_ROI14": {"0": 13, "1": 120, "2": 339, "3": 85, "4": 17, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 2, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "85_ROI9": {"0": 27, "1": 21, "2": 91, "3": 0, "4": 1, "5": 0, "6": 18, "7": 3, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 1, "16": 0, "17": 0, "label": 1}, "85_ROI11": {"0": 3, "1": 217, "2": 14, "3": 0, "4": 0, "5": 0, "6": 1, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "85_ROI12": {"0": 0, "1": 69, "2": 0, "3": 0, "4": 0, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "85_ROI14": {"0": 2, "1": 221, "2": 4, "3": 0, "4": 0, "5": 0, "6": 0, "7": 1, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "85_ROI15": {"0": 57, "1": 9, "2": 23, "3": 14, "4": 1, "5": 20, "6": 5, "7": 0, "8": 4, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "85_ROI16": {"0": 8, "1": 197, "2": 26, "3": 1, "4": 0, "5": 5, "6": 0, "7": 0, "8": 0, "9": 2, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "85_ROI18": {"0": 8, "1": 217, "2": 15, "3": 0, "4": 1, "5": 4, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "85_ROI19": {"0": 0, "1": 42, "2": 1, "3": 0, "4": 0, "5": 1, "6": 0, "7": 242, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "85_ROI20": {"0": 2, "1": 19, "2": 28, "3": 1, "4": 12, "5": 1, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "85_ROI21": {"0": 0, "1": 10, "2": 2, "3": 0, "4": 0, "5": 0, "6": 0, "7": 74, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "85_ROI22": {"0": 2, "1": 145, "2": 43, "3": 0, "4": 8, "5": 0, "6": 1, "7": 0, "8": 2, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "85_ROI1": {"0": 0, "1": 163, "2": 0, "3": 0, "4": 0, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "36_ROI9": {"0": 2, "1": 21, "2": 210, "3": 357, "4": 3, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 1, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "36_ROI10": {"0": 0, "1": 21, "2": 29, "3": 37, "4": 8, "5": 0, "6": 0, "7": 0, "8": 3, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 1, "17": 0, "label": 1}, "36_ROI11": {"0": 4, "1": 3, "2": 21, "3": 134, "4": 0, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "36_ROI12": {"0": 1, "1": 2, "2": 87, "3": 158, "4": 0, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "36_ROI14": {"0": 7, "1": 44, "2": 319, "3": 359, "4": 20, "5": 0, "6": 0, "7": 0, "8": 2, "9": 1, "10": 3, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 1, "17": 0, "label": 1}, "36_ROI15": {"0": 0, "1": 1, "2": 13, "3": 97, "4": 0, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "36_ROI16": {"0": 0, "1": 2, "2": 31, "3": 38, "4": 0, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "9_ROI9": {"0": 91, "1": 6, "2": 78, "3": 0, "4": 7, "5": 11, "6": 3, "7": 0, "8": 277, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "9_ROI10": {"0": 260, "1": 2, "2": 74, "3": 2, "4": 11, "5": 31, "6": 0, "7": 0, "8": 72, "9": 1, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "9_ROI11": {"0": 462, "1": 19, "2": 93, "3": 1, "4": 7, "5": 12, "6": 2, "7": 0, "8": 41, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "9_ROI12": {"0": 340, "1": 8, "2": 236, "3": 6, "4": 5, "5": 22, "6": 1, "7": 0, "8": 12, "9": 3, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "8_ROI1": {"0": 5, "1": 15, "2": 47, "3": 22, "4": 0, "5": 5, "6": 0, "7": 0, "8": 515, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "8_ROI2": {"0": 124, "1": 33, "2": 212, "3": 185, "4": 21, "5": 50, "6": 1, "7": 2, "8": 4, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "8_ROI3": {"0": 45, "1": 54, "2": 177, "3": 48, "4": 1, "5": 13, "6": 5, "7": 1, "8": 9, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "33_ROI9": {"0": 33, "1": 13, "2": 24, "3": 0, "4": 47, "5": 23, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "33_ROI10": {"0": 122, "1": 10, "2": 59, "3": 0, "4": 6, "5": 5, "6": 1, "7": 0, "8": 0, "9": 1, "10": 0, "11": 1, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "33_ROI11": {"0": 31, "1": 5, "2": 54, "3": 2, "4": 1, "5": 121, "6": 2, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "33_ROI12": {"0": 31, "1": 6, "2": 39, "3": 0, "4": 10, "5": 146, "6": 1, "7": 0, "8": 0, "9": 0, "10": 8, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "33_ROI13": {"0": 0, "1": 10, "2": 140, "3": 0, "4": 7, "5": 0, "6": 0, "7": 0, "8": 25, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 1, "16": 0, "17": 0, "label": 0}, "33_ROI14": {"0": 20, "1": 125, "2": 88, "3": 0, "4": 7, "5": 5, "6": 0, "7": 2, "8": 0, "9": 0, "10": 3, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "33_ROI15": {"0": 40, "1": 21, "2": 79, "3": 0, "4": 6, "5": 1, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "33_ROI16": {"0": 0, "1": 6, "2": 23, "3": 0, "4": 12, "5": 146, "6": 1, "7": 0, "8": 0, "9": 0, "10": 4, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "33_ROI17": {"0": 16, "1": 3, "2": 58, "3": 1, "4": 0, "5": 120, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "33_ROI18": {"0": 3, "1": 8, "2": 51, "3": 6, "4": 6, "5": 26, "6": 0, "7": 0, "8": 1, "9": 0, "10": 1, "11": 0, "12": 6, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "33_ROI19": {"0": 2, "1": 25, "2": 40, "3": 1, "4": 4, "5": 70, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "33_ROI20": {"0": 55, "1": 2, "2": 18, "3": 3, "4": 10, "5": 6, "6": 0, "7": 0, "8": 0, "9": 1, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "41_ROI9": {"0": 100, "1": 72, "2": 195, "3": 174, "4": 39, "5": 0, "6": 9, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "41_ROI10": {"0": 77, "1": 55, "2": 185, "3": 96, "4": 1, "5": 1, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "41_ROI11": {"0": 9, "1": 42, "2": 51, "3": 88, "4": 1, "5": 1, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "41_ROI12": {"0": 4, "1": 8, "2": 17, "3": 13, "4": 0, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "41_ROI13": {"0": 6, "1": 6, "2": 55, "3": 20, "4": 0, "5": 1, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "41_ROI14": {"0": 109, "1": 34, "2": 161, "3": 39, "4": 0, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 8, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "41_ROI15": {"0": 7, "1": 0, "2": 37, "3": 8, "4": 7, "5": 2, "6": 10, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 1, "17": 0, "label": 0}, "41_ROI16": {"0": 449, "1": 7, "2": 3, "3": 40, "4": 0, "5": 10, "6": 4, "7": 0, "8": 2, "9": 1, "10": 0, "11": 0, "12": 95, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "38_ROI9": {"0": 1, "1": 46, "2": 300, "3": 11, "4": 6, "5": 11, "6": 0, "7": 0, "8": 0, "9": 0, "10": 193, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "38_ROI10": {"0": 26, "1": 63, "2": 197, "3": 0, "4": 9, "5": 5, "6": 1, "7": 4, "8": 0, "9": 0, "10": 47, "11": 14, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "38_ROI11": {"0": 5, "1": 109, "2": 187, "3": 4, "4": 14, "5": 4, "6": 0, "7": 0, "8": 0, "9": 0, "10": 3, "11": 9, "12": 0, "13": 0, "14": 0, "15": 0, "16": 1, "17": 0, "label": 0}, "38_ROI12": {"0": 44, "1": 138, "2": 151, "3": 0, "4": 3, "5": 29, "6": 0, "7": 0, "8": 0, "9": 0, "10": 127, "11": 6, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "38_ROI13": {"0": 248, "1": 61, "2": 162, "3": 0, "4": 57, "5": 14, "6": 9, "7": 0, "8": 0, "9": 0, "10": 3, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 1, "17": 0, "label": 0}, "45_ROI9": {"0": 43, "1": 121, "2": 27, "3": 0, "4": 6, "5": 1, "6": 0, "7": 2, "8": 0, "9": 0, "10": 1, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "45_ROI10": {"0": 0, "1": 5, "2": 1, "3": 0, "4": 0, "5": 0, "6": 0, "7": 105, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "45_ROI11": {"0": 15, "1": 66, "2": 28, "3": 0, "4": 3, "5": 0, "6": 0, "7": 487, "8": 1, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 1, "15": 0, "16": 0, "17": 0, "label": 1}, "45_ROI12": {"0": 4, "1": 117, "2": 26, "3": 0, "4": 20, "5": 0, "6": 0, "7": 81, "8": 1, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "45_ROI13": {"0": 24, "1": 460, "2": 64, "3": 1, "4": 2, "5": 0, "6": 6, "7": 0, "8": 1, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "45_ROI14": {"0": 4, "1": 249, "2": 3, "3": 0, "4": 0, "5": 0, "6": 0, "7": 8, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "45_ROI15": {"0": 3, "1": 634, "2": 22, "3": 0, "4": 0, "5": 0, "6": 0, "7": 6, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "45_ROI16": {"0": 32, "1": 422, "2": 17, "3": 0, "4": 3, "5": 0, "6": 4, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "76_ROI10": {"0": 0, "1": 5, "2": 19, "3": 0, "4": 4, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 1, "17": 0, "label": 0}, "76_ROI12": {"0": 1, "1": 23, "2": 56, "3": 0, "4": 0, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "76_ROI13": {"0": 10, "1": 64, "2": 39, "3": 2, "4": 7, "5": 5, "6": 1, "7": 192, "8": 1, "9": 0, "10": 13, "11": 5, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 2, "label": 0}, "76_ROI14": {"0": 56, "1": 24, "2": 307, "3": 12, "4": 2, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 1, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "76_ROI15": {"0": 81, "1": 39, "2": 51, "3": 44, "4": 41, "5": 2, "6": 1, "7": 0, "8": 0, "9": 0, "10": 0, "11": 1, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "76_ROI16": {"0": 154, "1": 5, "2": 183, "3": 5, "4": 18, "5": 5, "6": 2, "7": 0, "8": 7, "9": 0, "10": 0, "11": 1, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "76_ROI17": {"0": 11, "1": 7, "2": 104, "3": 14, "4": 0, "5": 16, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "76_ROI18": {"0": 92, "1": 6, "2": 88, "3": 6, "4": 0, "5": 6, "6": 0, "7": 5, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "76_ROI19": {"0": 21, "1": 8, "2": 11, "3": 0, "4": 6, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "76_ROI20": {"0": 73, "1": 21, "2": 1501, "3": 70, "4": 0, "5": 2, "6": 1, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "79_ROI9": {"0": 12, "1": 51, "2": 695, "3": 59, "4": 0, "5": 19, "6": 2, "7": 0, "8": 34, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 2, "15": 0, "16": 0, "17": 0, "label": 1}, "79_ROI10": {"0": 17, "1": 86, "2": 242, "3": 142, "4": 5, "5": 11, "6": 2, "7": 0, "8": 76, "9": 1, "10": 0, "11": 0, "12": 0, "13": 0, "14": 6, "15": 0, "16": 0, "17": 0, "label": 1}, "79_ROI11": {"0": 3, "1": 0, "2": 46, "3": 94, "4": 0, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "79_ROI12": {"0": 0, "1": 1, "2": 199, "3": 24, "4": 3, "5": 2, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "79_ROI13": {"0": 1, "1": 8, "2": 33, "3": 4, "4": 17, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 1, "17": 0, "label": 1}, "79_ROI14": {"0": 0, "1": 32, "2": 79, "3": 5, "4": 1, "5": 4, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "79_ROI15": {"0": 2, "1": 83, "2": 219, "3": 149, "4": 1, "5": 1, "6": 0, "7": 11, "8": 0, "9": 0, "10": 5, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "79_ROI16": {"0": 62, "1": 7, "2": 96, "3": 167, "4": 1, "5": 3, "6": 2, "7": 0, "8": 18, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "79_ROI17": {"0": 9, "1": 30, "2": 74, "3": 6, "4": 1, "5": 3, "6": 0, "7": 0, "8": 1, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 2, "15": 0, "16": 0, "17": 0, "label": 1}, "79_ROI18": {"0": 2, "1": 40, "2": 61, "3": 9, "4": 11, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "79_ROI19": {"0": 0, "1": 1, "2": 20, "3": 0, "4": 0, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "63_ROI10": {"0": 0, "1": 4, "2": 8, "3": 4, "4": 0, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 106, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "63_ROI11": {"0": 1, "1": 19, "2": 49, "3": 0, "4": 17, "5": 0, "6": 0, "7": 0, "8": 0, "9": 8, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "63_ROI12": {"0": 0, "1": 18, "2": 42, "3": 0, "4": 6, "5": 1, "6": 0, "7": 0, "8": 0, "9": 39, "10": 1, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "63_ROI13": {"0": 0, "1": 4, "2": 79, "3": 75, "4": 0, "5": 1, "6": 0, "7": 0, "8": 0, "9": 0, "10": 5, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "63_ROI14": {"0": 6, "1": 1, "2": 54, "3": 0, "4": 7, "5": 11, "6": 0, "7": 0, "8": 0, "9": 2, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "63_ROI15": {"0": 10, "1": 6, "2": 26, "3": 0, "4": 7, "5": 5, "6": 0, "7": 0, "8": 0, "9": 4, "10": 0, "11": 1, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "63_ROI16": {"0": 0, "1": 7, "2": 41, "3": 0, "4": 8, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "67_ROI9": {"0": 44, "1": 12, "2": 35, "3": 131, "4": 4, "5": 2, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 23, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "67_ROI10": {"0": 61, "1": 112, "2": 99, "3": 18, "4": 3, "5": 4, "6": 1, "7": 2, "8": 0, "9": 0, "10": 0, "11": 1, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "67_ROI11": {"0": 98, "1": 18, "2": 112, "3": 1, "4": 12, "5": 0, "6": 4, "7": 1, "8": 0, "9": 0, "10": 0, "11": 3, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "67_ROI12": {"0": 191, "1": 39, "2": 184, "3": 2, "4": 33, "5": 5, "6": 3, "7": 0, "8": 0, "9": 1, "10": 0, "11": 23, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "67_ROI14": {"0": 192, "1": 53, "2": 162, "3": 0, "4": 31, "5": 0, "6": 0, "7": 1, "8": 0, "9": 0, "10": 0, "11": 6, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "67_ROI15": {"0": 81, "1": 249, "2": 76, "3": 1, "4": 8, "5": 0, "6": 1, "7": 2, "8": 1, "9": 0, "10": 0, "11": 1, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "67_ROI16": {"0": 107, "1": 44, "2": 92, "3": 3, "4": 15, "5": 29, "6": 1, "7": 0, "8": 2, "9": 0, "10": 1, "11": 2, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "67_ROI17": {"0": 145, "1": 56, "2": 145, "3": 0, "4": 32, "5": 17, "6": 6, "7": 1, "8": 4, "9": 0, "10": 5, "11": 16, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "67_ROI18": {"0": 55, "1": 23, "2": 45, "3": 2, "4": 7, "5": 24, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 1, "17": 0, "label": 1}, "52_ROI9": {"0": 5, "1": 6, "2": 55, "3": 219, "4": 1, "5": 1, "6": 0, "7": 0, "8": 1, "9": 1, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "52_ROI10": {"0": 10, "1": 13, "2": 152, "3": 75, "4": 18, "5": 6, "6": 1, "7": 0, "8": 2, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 2, "17": 0, "label": 1}, "52_ROI11": {"0": 14, "1": 167, "2": 317, "3": 41, "4": 13, "5": 39, "6": 0, "7": 0, "8": 1, "9": 1, "10": 5, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "52_ROI12": {"0": 39, "1": 49, "2": 47, "3": 254, "4": 6, "5": 10, "6": 1, "7": 0, "8": 0, "9": 8, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "52_ROI13": {"0": 4, "1": 3, "2": 50, "3": 181, "4": 7, "5": 5, "6": 0, "7": 0, "8": 0, "9": 2, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 1, "16": 0, "17": 0, "label": 1}, "53_ROI10": {"0": 1, "1": 1, "2": 51, "3": 1, "4": 7, "5": 0, "6": 0, "7": 0, "8": 2, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "53_ROI11": {"0": 56, "1": 9, "2": 7, "3": 3, "4": 21, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 1, "17": 0, "label": 0}, "53_ROI12": {"0": 446, "1": 49, "2": 75, "3": 3, "4": 84, "5": 0, "6": 12, "7": 0, "8": 0, "9": 4, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "53_ROI14": {"0": 84, "1": 15, "2": 126, "3": 8, "4": 76, "5": 9, "6": 1, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "55_ROI9": {"0": 27, "1": 11, "2": 92, "3": 0, "4": 19, "5": 247, "6": 2, "7": 0, "8": 17, "9": 1, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 1, "16": 0, "17": 0, "label": 0}, "55_ROI10": {"0": 19, "1": 14, "2": 15, "3": 3, "4": 13, "5": 425, "6": 0, "7": 0, "8": 1, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "55_ROI11": {"0": 5, "1": 18, "2": 4, "3": 0, "4": 4, "5": 127, "6": 0, "7": 0, "8": 1, "9": 0, "10": 0, "11": 3, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "55_ROI12": {"0": 8, "1": 0, "2": 1, "3": 0, "4": 7, "5": 112, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "55_ROI13": {"0": 5, "1": 0, "2": 6, "3": 1, "4": 1, "5": 210, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "55_ROI14": {"0": 3, "1": 4, "2": 8, "3": 1, "4": 13, "5": 122, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "55_ROI15": {"0": 18, "1": 1, "2": 19, "3": 0, "4": 2, "5": 200, "6": 1, "7": 0, "8": 1, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "55_ROI16": {"0": 4, "1": 7, "2": 4, "3": 6, "4": 1, "5": 112, "6": 1, "7": 0, "8": 1, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 0}, "65_ROI9": {"0": 0, "1": 45, "2": 573, "3": 10, "4": 2, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 1, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "65_ROI10": {"0": 1, "1": 10, "2": 137, "3": 0, "4": 2, "5": 0, "6": 0, "7": 0, "8": 1, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "65_ROI11": {"0": 0, "1": 65, "2": 154, "3": 1, "4": 0, "5": 0, "6": 0, "7": 18, "8": 0, "9": 0, "10": 2, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "65_ROI13": {"0": 2, "1": 6, "2": 203, "3": 0, "4": 1, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "65_ROI14": {"0": 0, "1": 0, "2": 26, "3": 0, "4": 0, "5": 0, "6": 0, "7": 82, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "65_ROI15": {"0": 0, "1": 95, "2": 115, "3": 0, "4": 15, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 3, "17": 0, "label": 1}, "65_ROI16": {"0": 0, "1": 9, "2": 49, "3": 0, "4": 3, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "65_ROI17": {"0": 0, "1": 10, "2": 111, "3": 2, "4": 6, "5": 0, "6": 1, "7": 0, "8": 1, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "65_ROI18": {"0": 7, "1": 14, "2": 63, "3": 5, "4": 1, "5": 0, "6": 0, "7": 0, "8": 0, "9": 1, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "65_ROI19": {"0": 10, "1": 5, "2": 87, "3": 0, "4": 0, "5": 4, "6": 0, "7": 0, "8": 0, "9": 1, "10": 0, "11": 0, "12": 0, "13": 0, "14": 1, "15": 0, "16": 0, "17": 0, "label": 1}, "65_ROI20": {"0": 32, "1": 12, "2": 57, "3": 2, "4": 38, "5": 1, "6": 1, "7": 0, "8": 0, "9": 2, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "65_ROI21": {"0": 14, "1": 24, "2": 145, "3": 1, "4": 1, "5": 0, "6": 6, "7": 0, "8": 0, "9": 1, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "68_ROI9": {"0": 7, "1": 219, "2": 15, "3": 0, "4": 2, "5": 0, "6": 1, "7": 377, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 2, "15": 0, "16": 0, "17": 0, "label": 1}, "68_ROI10": {"0": 145, "1": 65, "2": 212, "3": 1, "4": 9, "5": 2, "6": 1, "7": 3, "8": 2, "9": 1, "10": 0, "11": 0, "12": 0, "13": 0, "14": 38, "15": 1, "16": 0, "17": 0, "label": 1}, "68_ROI11": {"0": 0, "1": 47, "2": 6, "3": 0, "4": 0, "5": 0, "6": 0, "7": 588, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "label": 1}, "68_ROI12": {"0": 2, "1": 200, "2": 35, "3": 0, "4": 0, "5": 2, "6": 0, "7": 236, "8": 0, "9": 0, "10": 0, "11": 1, "12": 0, "13": 0, "14": 1, "15": 0, "16": 0, "17": 0, "label": 1}, "68_ROI13": {"0": 114, "1": 10, "2": 55, "3": 2, "4": 7, "5": 17, "6": 5, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 4, "15": 0, "16": 0, "17": 0, "label": 1}}'''

df = pd.read_json(data_res, orient ='index')

In [13]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,label
66_ROI9,13,234,168,10,0,2,0,21,0,0,0,1,0,0,2,0,0,0,1
66_ROI10,21,495,33,10,0,20,0,88,0,0,1,0,0,0,0,0,0,0,1
77_ROI10,16,269,45,1,29,1,7,39,6,0,31,1,0,0,0,0,0,0,0
77_ROI11,264,8,57,104,1,0,11,0,1,4,0,2,0,0,0,0,0,0,0
12_ROI9,171,42,32,4,5,330,1,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68_ROI9,42,606,55,0,7,3,3,335,0,0,0,0,0,0,6,0,0,0,1
68_ROI10,348,58,175,1,13,10,10,0,1,2,0,4,0,1,18,0,0,0,1
68_ROI11,15,190,49,0,0,3,0,702,0,0,0,0,0,0,0,0,0,0,1
68_ROI12,77,342,186,0,8,9,0,146,10,0,0,0,0,0,16,1,0,0,1


In [14]:
df.to_csv('./Global_count_domain_ratio.csv')